# FUSION FINALE DES JEUX DE DONNEES

In [1]:
from utils.io import dataframe_viewer, files_search, data_merger, data_validation, data_overview, \
gen_id_from_ech, na_line_drop

import re, os
import pandas as pd
from definitions import ROOT_DIR

In [2]:
def create_df(files, check_position=True, verbose=True): # find another name for this function
    """
    create dataframes from files and test if they contain position informations
    files: list of files name
    """
    dfs = []
    i = 0
    for f in files:
        i += 1
        df = pd.read_csv(f, delimiter=',')
        if check_position:
            if 'X' in df.columns:
                df['X'] = df['X'].apply(lambda x: x.replace(',','.') if isinstance(x, str) else x)
                df['X'] = df['X'].astype('float64')
            if 'Y' in df.columns:
                df['Y'] = df['Y'].apply(lambda x: x.replace(',','.') if isinstance(x, str) else x)
                df['Y'] = df['Y'].astype('float64')
            if 'Z' in df.columns:
                df['Z'] = df['Z'].apply(lambda x: x.replace(',','.') if isinstance(x, str) else x)
                df['Z'] = df['Z'].astype('float64')
        
        df = na_line_drop(df, line_non_na=1)
        dfs.append(df)
        
        if verbose:
            if 'X' in list(df.columns): msg = ' --> Coordinates'
            else: msg = ' --> No coordinates'

            print(f"df{i} : {msg}")
    
            
    return dfs

In [3]:
csv_data_dir = ROOT_DIR + '/CF_data/Result_traitem/organisation/'
save_dir = csv_data_dir + '../fusion_finale/'

In [4]:
os.system(f"rm -rf {save_dir}") 
os.system(f"mkdir {save_dir}") 

0

In [5]:
sufx = ['sup', 'prof', 'inf', '/\dM(\*)?']
prefx = ['eau forage ']
id_reg = '\s*(?P<id>(?:^canne |Piezair )*\w*\d+\w*)\s*'

# Collecte des fichiers

In [6]:
# create my dictionary structure to retrieve good files (Keynames !!!)
files_dict={'Borehole':0,'Litho':0,'Equipm':0,'Measure':0,'Sample':0,'Unknow':0}
data_dict={'Borehole':0,'Litho':0,'Equipm':0,'Measure':0,'Sample':0,'Unknow':0}

In [7]:
files_search(csv_data_dir, files_dict, prefix='source', details=False)

Borehole  	:  15
Litho  	:  4
Equipm  	:  5
Measure  	:  13
Sample  	:  13
Unknow  	:  1


In [8]:
how=['inner', 'outer', 'left', 'right']
view = False

## Forages

In [9]:
key='Borehole'
dataset = pd.DataFrame()
print(len(files_dict[key]), 'files')

15 files


In [10]:
files_dict[key]

['/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Forage_Pilote/source_merge/source_Boreholes.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Memoris_seafile/source_merge/source_Boreholes.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Phase_1_Memoris/source_merge/source_Boreholes.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Phase_2_Memoris/source_merge/source_Boreholes.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Prof_contact_sol_forage/source_merge/source_Boreholes.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Siterem_Ext_Pilote/source_merge/source_Boreholes.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Siterem_Pilote/source_merge/source_Boreholes.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Siterem_Result_Sol/source_merge/source_Boreholes.csv',
 '/home/yanathan/Projects/GSD

In [11]:
data_overview(files_dict[key])

Same files:[]
Files with coordinates:[1, 2, 3, 4, 8, 9, 10, 12, 14]
Files without coordinates:[0, 5, 6, 7, 11, 13]


#### $\color{green}{\textbf{Lecture et fusion}}$

In [12]:
file1= csv_data_dir + 'Memoris_seafile/source_merge/source_Boreholes.csv' # 1
file2= csv_data_dir + 'Phase_1_Memoris/source_merge/source_Boreholes.csv' # 2

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 57, columns : 8, Unique values on cols: {'ID': 57}
Rows : 14, columns : 7, Unique values on cols: {'ID': 14}


(None, None)

In [13]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID'], dist_max=1)

In [14]:
dataset = mdf.copy()

In [15]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 69


#### $\color{green}{\textbf{Lecture et fusion}}$

In [16]:
file1= csv_data_dir + 'Phase_2_Memoris/source_merge/source_Boreholes.csv' # 3
file2= csv_data_dir + 'Prof_contact_sol_forage/source_merge/source_Boreholes.csv' # 4

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 22, columns : 8, Unique values on cols: {'ID': 22}
Rows : 8, columns : 6, Unique values on cols: {'ID': 8}


(None, None)

In [17]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID'], dist_max=1)

In [18]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID'], dist_max=1)

In [19]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 77


#### $\color{green}{\textbf{Lecture et fusion}}$

In [20]:
file1= csv_data_dir + 'database_Memoris3/source_merge/source_Boreholes.csv' # 8
file2= csv_data_dir + 'donnees_terrain_2019/source_merge/source_Boreholes.csv' # 9

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 520, columns : 20, Unique values on cols: {'ID': 518}
Rows : 25, columns : 17, Unique values on cols: {'ID': 25}


(None, None)

In [21]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID'], dist_max=1)

In [22]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID'], dist_max=1)

Conflict values present. Please resolve this manually !


In [23]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'Type_y':list(conflict_df.index), 'Long_for_y':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [24]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 605


#### $\color{green}{\textbf{Lecture et fusion}}$

In [25]:
file1= csv_data_dir + 'profils_sols_donnees_forages/source_merge/source_Boreholes.csv' # 11
file2= csv_data_dir + 'vUmons_logsFor/source_merge/source_Boreholes.csv' # 13

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 27, columns : 19, Unique values on cols: {'ID': 27}
Rows : 78, columns : 9, Unique values on cols: {'ID': 77}


(None, None)

In [26]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID'], dist_max=1)

In [27]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID'], dist_max=1)

Conflict values present. Please resolve this manually !


In [28]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'Date_for_y':list(conflict_df.index), 'Type_y':list(conflict_df.index), 
                            'Long_for_y':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [29]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 677


#### $\color{green}{\textbf{Lecture et fusion}}$

In [30]:
file1= csv_data_dir + 'Forage_Pilote/source_merge/source_Boreholes.csv' # 0
file2= csv_data_dir + 'Siterem_Ext_Pilote/source_merge/source_Boreholes.csv' # 5

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

14 NaN lines dropped
df1 :  --> No coordinates
df2 :  --> No coordinates
Rows : 58, columns : 5, Unique values on cols: {'ID': 58}
Rows : 10, columns : 5, Unique values on cols: {'ID': 6}


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


(None, None)

In [31]:
df2['ID'] = df2['ID'].astype('object')

In [32]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID'], dist_max=1)

In [33]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID'], dist_max=1)

Conflict values present. Please resolve this manually !


In [34]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'Z_x':list(conflict_df.index), 'Type_x':list(conflict_df.index), 
                            'Long_for_x':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [35]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 701


#### $\color{green}{\textbf{Lecture et fusion}}$

In [36]:
file1= csv_data_dir + 'Siterem_Pilote/source_merge/source_Boreholes.csv' # 6
file2= csv_data_dir + 'Siterem_Result_Sol/source_merge/source_Boreholes.csv' # 7

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

22 NaN lines dropped
df1 :  --> No coordinates
23 NaN lines dropped
df2 :  --> No coordinates
Rows : 0, columns : 4, Unique values on cols: {'ID': 0}
Rows : 0, columns : 4, Unique values on cols: {'ID': 0}


(None, None)

In [37]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID'], dist_max=1)

In [38]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID'], dist_max=1)

In [39]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 701


#### $\color{green}{\textbf{Lecture et fusion}}$

In [40]:
file1= csv_data_dir + 'observ_terrain/source_merge/source_Boreholes.csv' # 10
file2= csv_data_dir + 'result_sol_ext_pilote/source_merge/source_Boreholes.csv' # 12

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

5 NaN lines dropped
df1 :  --> No coordinates
19 NaN lines dropped
df2 :  --> No coordinates
Rows : 22, columns : 6, Unique values on cols: {'ID': 22}
Rows : 0, columns : 4, Unique values on cols: {'ID': 0}


(None, None)

In [41]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID'], dist_max=1)

In [42]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID'], dist_max=1)

Conflict values present. Please resolve this manually !


In [43]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'Long_pz_sol_x':[18], 'Long_pz_sol_y':[i for i in conflict_df.index if i not in [18]], 
                            'Rmq_y':list(conflict_df.index), 'Type_x':list(conflict_df.index),})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [44]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 701


#### $\color{green}{\textbf{Lecture et fusion}}$

In [45]:
file1= csv_data_dir + 'coord_ext_pilote/source_merge/source_Boreholes.csv' # 10

df1 = create_df([file1])[0]
dataframe_viewer(df1, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
Rows : 78, columns : 5, Unique values on cols: {'ID': 78}


In [46]:
data = df1
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['X','Y'], dist_max=1)

Conflict values present. Please resolve this manually !


In [47]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'ID_x':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [48]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 763


####  $\color{red}{\textbf{Sauvegarde du jeu de données}}$

In [49]:
data_dict['Borehole'] = dataset.copy()
dataframe_viewer(dataset, rows=3, un_val=['ID','ID_ech'])

Rows : 763, columns : 28, Unique values on cols: {'ID': 742, 'ID_ech': 'NA'}


interactive(children=(IntSlider(value=3, description='rows', max=763, min=3, readout=False), IntSlider(value=1…

In [50]:
dataset.to_csv(save_dir + 'Boreholes.csv', index=False)

###  ====================================================

# Lithologies

In [51]:
key='Litho'
dataset = pd.DataFrame()
print(len(files_dict[key]), 'files')

4 files


In [52]:
files_dict[key]

['/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/database_Memoris3/source_merge/source_Lithologies.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/donnees_terrain_2019/source_merge/source_Lithologies.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/profils_sols_donnees_forages/source_merge/source_Lithologies.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/vUmons_logsFor/source_merge/source_Lithologies.csv']

In [53]:
data_overview(files_dict[key])

Same files:[]
Files with coordinates:[0, 2, 3]
Files without coordinates:[1]


#### $\color{green}{\textbf{Lecture et fusion}}$

In [54]:
file1= csv_data_dir + 'database_Memoris3/source_merge/source_Lithologies.csv' # 0
file2= csv_data_dir + 'profils_sols_donnees_forages/source_merge/source_Lithologies.csv' # 2

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 1627, columns : 9, Unique values on cols: {'ID': 298}
Rows : 55, columns : 9, Unique values on cols: {'ID': 25}


(None, None)

In [55]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID', 'Litho_top', 'Litho_base'], dist_max=1)

In [56]:
dataset = mdf.copy()

In [57]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 1682


#### $\color{green}{\textbf{Lecture et fusion}}$

In [58]:
file1= csv_data_dir + 'donnees_terrain_2019/source_merge/source_Lithologies.csv' # 1
file2= csv_data_dir + 'vUmons_logsFor/source_merge/source_Lithologies.csv' # 3

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> No coordinates
df2 :  --> Coordinates
Rows : 93, columns : 9, Unique values on cols: {'ID': 25}
Rows : 112, columns : 9, Unique values on cols: {'ID': 71}


(None, None)

In [59]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID', 'Litho_top', 'Litho_base'], dist_max=1)

In [60]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID', 'Litho_top', 'Litho_base'], dist_max=1)

In [61]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 1887


####  $\color{red}{\textbf{Sauvegarde du jeu de données}}$

In [62]:
data_dict['Litho'] = dataset.copy()
dataframe_viewer(dataset, rows=3, un_val=['ID','ID_ech'])

Rows : 1887, columns : 12, Unique values on cols: {'ID': 418, 'ID_ech': 'NA'}


interactive(children=(IntSlider(value=3, description='rows', max=1887, min=3, readout=False), IntSlider(value=…

In [63]:
dataset.to_csv(save_dir + 'Lithologies.csv', index=False)

###  ====================================================

# Echantillons

In [64]:
key='Sample'
dataset = pd.DataFrame()
print(len(files_dict[key]), 'files')

13 files


In [65]:
files_dict[key]

['/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Container_phyto/source_merge/source_Samples.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Liste_XY/source_merge/source_Samples.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Memoris_seafile/source_merge/source_Samples.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Phase_1_Memoris/source_merge/source_Samples.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Phase_2_Memoris/source_merge/source_Samples.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Siterem_Ext_Pilote/source_merge/source_Samples.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Siterem_Pilote/source_merge/source_Samples.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Siterem_Result_Sol/source_merge/source_Samples.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/org

In [66]:
data_overview(files_dict[key])

Same files:[]
Files with coordinates:[1, 2, 3, 4, 8, 10, 12]
Files without coordinates:[0, 5, 6, 7, 9, 11]


#### $\color{green}{\textbf{Lecture et fusion}}$

In [67]:
file1= csv_data_dir + 'Memoris_seafile/source_merge/source_Samples.csv' # 2
file2= csv_data_dir + 'Liste_XY/source_merge/source_Samples.csv' # 1

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 105, columns : 91, Unique values on cols: {'ID': 60}
Rows : 254, columns : 6, Unique values on cols: {'ID': 254}


(None, None)

In [68]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID', 'ID_ech'], dist_max=1)

In [69]:
dataset = mdf.copy()

In [70]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 358


#### $\color{green}{\textbf{Lecture et fusion}}$

In [71]:
file1= csv_data_dir +  'Phase_1_Memoris/source_merge/source_Samples.csv' # 3
file2= csv_data_dir + 'Phase_2_Memoris/source_merge/source_Samples.csv' # 4

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 46, columns : 84, Unique values on cols: {'ID': 29}
Rows : 36, columns : 83, Unique values on cols: {'ID': 22}


(None, None)

In [72]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

0 Date_ech
0 ID_ech
0 ID
1 Date_ech
1 ID_ech
1 ID


In [73]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID', 'ID_ech', 'Date_ech'], dist_max=1)

In [74]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID', 'ID_ech', 'Date_ech'], dist_max=1)

Conflict values present. Please resolve this manually !


In [75]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'C16-C21_y':list(conflict_df.index), 'C21-C35_y':list(conflict_df.index), 
                            'C12-C16_y':list(conflict_df.index), 'Fract_2+_y':list(conflict_df.index), 
                            'C10-C12_y':list(conflict_df.index), 'HC_tot_C10-C35_y':list(conflict_df.index), 
                            'Fract_2_y':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [76]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 404


#### $\color{green}{\textbf{Lecture et fusion}}$

In [77]:
file1= csv_data_dir +  'database_Memoris3/source_merge/source_Samples.csv' # 8
file2= csv_data_dir +  'profils_sols_donnees_forages/source_merge/source_Samples.csv' # 10

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
2 NaN lines dropped
df2 :  --> Coordinates
Rows : 1583, columns : 197, Unique values on cols: {'ID': 470}
Rows : 39, columns : 9, Unique values on cols: {'ID': 25}


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


(None, None)

In [78]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

0 Date_ech
0 Date_for
0 ID
0 ID_ech
1 Date_for
1 ID
1 ID_ech


In [79]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID', 'ID_ech'], dist_max=1)

In [80]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID', 'ID_ech', 'Date_ech'], dist_max=1)

Conflict values present. Please resolve this manually !


In [81]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'Nappe_x':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [82]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 1975


#### $\color{green}{\textbf{Lecture et fusion}}$

In [83]:
file1= csv_data_dir +  'Container_phyto/source_merge/source_Samples.csv' # 0
file2= csv_data_dir +  'vUmons_logsFor/source_merge/source_Samples.csv' # 12

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

1 NaN lines dropped
df1 :  --> No coordinates
df2 :  --> Coordinates
Rows : 5, columns : 82, Unique values on cols: {'ID': 'NA'}
Rows : 45, columns : 76, Unique values on cols: {'ID': 29}


(None, None)

In [84]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

0 Date_ech
0 ID_ech
1 Date_ech
1 ID_ech
1 ID


In [85]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID_ech', 'Date_ech'], dist_max=1)

In [86]:
mdf = gen_id_from_ech(mdf, id_ech_col='ID_ech', suffixes=sufx, prefixes=prefx, capture_regex=id_reg)

In [87]:
data = mdf
col = 'ID_ech'
for i in data.index:
    v = data.loc[i, col]
    if not pd.isnull(v) and re.search('ech', v, re.I):
        data.loc[i, 'ID'] = 'F_' + re.sub(' |.','', v,re.I)
mdf = data.copy()

In [88]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID', 'ID_ech', 'Date_ech'], dist_max=1)

In [89]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 2025


#### $\color{green}{\textbf{Lecture et fusion}}$

In [90]:
file1= csv_data_dir +  'Siterem_Ext_Pilote/source_merge/source_Samples.csv' # 5
file2= csv_data_dir +  'Siterem_Pilote/source_merge/source_Samples.csv' # 6

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

1 NaN lines dropped
df1 :  --> No coordinates
df2 :  --> No coordinates
Rows : 43, columns : 97, Unique values on cols: {'ID': 6}
Rows : 95, columns : 97, Unique values on cols: {'ID': 12}


(None, None)

In [91]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

0 Date_ech
0 ID_ech
0 ID
1 Date_ech
1 ID_ech
1 ID


In [92]:
df1['ID_ech'] = df1['ID_ech'].astype('object')

In [93]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID_ech', 'Date_ech'], dist_max=1)

In [94]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID', 'ID_ech', 'Date_ech'], dist_max=1)

Conflict values present. Please resolve this manually !


In [95]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'C16-C21_x':list(conflict_df.index), 'C12-C16_x':list(conflict_df.index), 
                            'C10-C12_x':list(conflict_df.index), 'HC_tot_C10-C35_x':list(conflict_df.index), 
                            'C21-C35_x':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [96]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 2162


#### $\color{green}{\textbf{Lecture et fusion}}$

In [97]:
file1= csv_data_dir +  'Siterem_Result_Sol/source_merge/source_Samples.csv' # 7
file2= csv_data_dir +  'donnees_terrain_2019/source_merge/source_Samples.csv' # 9

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

1 NaN lines dropped
df1 :  --> No coordinates
df2 :  --> No coordinates
Rows : 59, columns : 82, Unique values on cols: {'ID': 23}
Rows : 70, columns : 9, Unique values on cols: {'ID': 22}


(None, None)

In [98]:
df2['ID'] = df2['ID'].astype('object')

In [99]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

0 Date_ech
0 ID_ech
0 ID
1 ID
1 ID_ech


In [100]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID', 'ID_ech'], dist_max=1)

In [101]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID', 'ID_ech', 'Date_ech'], dist_max=1)

In [102]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 2291


#### $\color{green}{\textbf{Lecture et fusion}}$

In [103]:
file1= csv_data_dir +  'result_sol_ext_pilote/source_merge/source_Samples.csv' # 11

df1 = create_df([file1])[0]
dataframe_viewer(df1, rows=3, un_val='ID', view=view)

df1 :  --> No coordinates
Rows : 53, columns : 85, Unique values on cols: {'ID': 19}


In [104]:
df_list = [df1]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

0 Date_ech
0 ID
0 ID_ech


In [105]:
data = df1
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID', 'ID_ech', 'Date_ech'], dist_max=1)

In [106]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 2305


####  $\color{red}{\textbf{Sauvegarde du jeu de données}}$

In [107]:
data_dict['Sample'] = dataset.copy()
dataframe_viewer(dataset, rows=3, un_val=['ID','ID_ech'])

Rows : 2305, columns : 219, Unique values on cols: {'ID': 819, 'ID_ech': 1878}


interactive(children=(IntSlider(value=3, description='rows', max=2305, min=3, readout=False), IntSlider(value=…

In [108]:
dataset.to_csv(save_dir + 'Samples.csv', index=False)

###  ====================================================

## Objets inconnus

In [109]:
key='Unknow'
dataset = pd.DataFrame()
print(len(files_dict[key]), 'files')

1 files


In [110]:
files_dict[key]

['/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/database_Memoris3/source_merge/source_Unknown.csv']

#### $\color{green}{\textbf{Lecture et fusion}}$

In [111]:
file1= csv_data_dir +  'database_Memoris3/source_merge/source_Unknown.csv' # 11

df1 = create_df([file1])[0]
dataframe_viewer(df1, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
Rows : 32, columns : 12, Unique values on cols: {'ID': 32}


In [112]:
dataset = df1.drop(columns='ID_date')

In [113]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 32


####  $\color{red}{\textbf{Sauvegarde du jeu de données}}$

In [114]:
data_dict['Unknow'] = dataset.copy()
dataframe_viewer(dataset, rows=3, un_val=['ID','ID_ech'])

Rows : 32, columns : 11, Unique values on cols: {'ID': 32, 'ID_ech': 'NA'}


interactive(children=(IntSlider(value=3, description='rows', max=32, min=3, readout=False), IntSlider(value=11…

In [115]:
dataset.to_csv(save_dir + 'Unknown.csv', index=False)

###  ====================================================

## Equipements

In [116]:
key='Equipm'
dataset = pd.DataFrame()
print(len(files_dict[key]), 'files')

5 files


In [117]:
files_dict[key]

['/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Memoris_seafile/source_merge/source_Equipments.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Phase_1_Memoris/source_merge/source_Equipments.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Phase_2_Memoris/source_merge/source_Equipments.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/donnees_terrain_2019/source_merge/source_Equipments.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/profils_sols_donnees_forages/source_merge/source_Equipments.csv']

In [118]:
data_overview(files_dict[key])

Same files:[]
Files with coordinates:[0, 1, 2, 4]
Files without coordinates:[3]


#### $\color{green}{\textbf{Lecture et fusion}}$

In [119]:
file1= csv_data_dir + 'Memoris_seafile/source_merge/source_Equipments.csv' # 0
file2= csv_data_dir + 'Phase_1_Memoris/source_merge/source_Equipments.csv' # 1

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

6 NaN lines dropped
df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 39, columns : 7, Unique values on cols: {'ID': 24}
Rows : 17, columns : 7, Unique values on cols: {'ID': 14}


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


(None, None)

In [120]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID'], dist_max=1)

In [121]:
dataset = mdf.copy()

In [122]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 54


#### $\color{green}{\textbf{Lecture et fusion}}$

In [123]:
file1= csv_data_dir + 'Phase_2_Memoris/source_merge/source_Equipments.csv' # 2
file1= csv_data_dir + 'profils_sols_donnees_forages/source_merge/source_Equipments.csv' # 4

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 36, columns : 8, Unique values on cols: {'ID': 12}
Rows : 17, columns : 7, Unique values on cols: {'ID': 14}


(None, None)

In [124]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

0 Date_for
0 ID
1 ID


In [125]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID'], dist_max=1)

In [126]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID'], dist_max=1)

In [127]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 96


#### $\color{green}{\textbf{Lecture et fusion}}$

In [128]:
file1= csv_data_dir + 'donnees_terrain_2019/source_merge/source_Equipments.csv' # 3

df1 = create_df([file1])[0]
dataframe_viewer(df1, rows=3, un_val='ID', view=view)

df1 :  --> No coordinates
Rows : 33, columns : 8, Unique values on cols: {'ID': 9}


In [129]:
df1['ID'] = df1['ID'].astype('object')

In [130]:
data = df1
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID'], dist_max=1)

In [131]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 129


####  $\color{red}{\textbf{Sauvegarde du jeu de données}}$

In [132]:
data_dict['Equipm'] = dataset.copy()
dataframe_viewer(dataset, rows=3, un_val=['ID'])

Rows : 129, columns : 14, Unique values on cols: {'ID': 57}


interactive(children=(IntSlider(value=3, description='rows', max=129, min=3, readout=False), IntSlider(value=1…

In [133]:
dataset.to_csv(save_dir + 'Equipments.csv', index=False)

###  ====================================================

# Mesures

In [134]:
key='Measure'
dataset = pd.DataFrame()
print(len(files_dict[key]), 'files')

13 files


In [135]:
files_dict[key]

['/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Container_phyto/source_merge/source_Measures.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Memoris_seafile/source_merge/source_Measures.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Phase_1_Memoris/source_merge/source_Measures.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Phase_2_Memoris/source_merge/source_Measures.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Siterem_Ext_Pilote/source_merge/source_Measures.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Siterem_Pilote/source_merge/source_Measures.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Siterem_Result_Sol/source_merge/source_Measures.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/database_Memoris3/source_merge/source_Measures.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/R

In [136]:
data_overview(files_dict[key])

Same files:[]
Files with coordinates:[1, 2, 3, 7, 10, 12]
Files without coordinates:[0, 4, 5, 6, 8, 9, 11]


In [137]:
#id_reg = '\s*(?P<id>(?:^canne |Piezair )*\w*\d+\w*)\s*' # default 
id_reg = '\s*(?P<id>(?:^canne |Piezair |Drain |Moni )*\w*\d+\w*)\s*'

#### $\color{green}{\textbf{Lecture et fusion}}$

In [138]:
file1= csv_data_dir + 'Memoris_seafile/source_merge/source_Measures.csv' #1
file2= csv_data_dir + 'Phase_1_Memoris/source_merge/source_Measures.csv' # 2

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 45, columns : 11, Unique values on cols: {'ID': 30}
Rows : 17, columns : 10, Unique values on cols: {'ID': 14}


(None, None)

In [139]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

0 Date_mes
0 ID_ech
0 ID
1 Date_mes
1 ID_ech
1 ID


In [140]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID','ID_ech','Date_mes'], dist_max=1)

In [141]:
dataset = mdf.copy()

In [142]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 60


#### $\color{green}{\textbf{Lecture et fusion}}$

In [143]:
file1= csv_data_dir + 'Phase_2_Memoris/source_merge/source_Measures.csv' # 3
file2= csv_data_dir + 'database_Memoris3/source_merge/source_Measures.csv' # 7

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 11, columns : 10, Unique values on cols: {'ID': 10}
Rows : 1734, columns : 33, Unique values on cols: {'ID': 470}


(None, None)

In [144]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

0 Date_mes
0 ID_ech
0 ID
1 Date_mes
1 Date_for
1 ID
1 Residu_perte_feu
1 ID_ech


In [145]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID','ID_ech','Date_mes'], dist_max=1)

In [146]:
datafr = mdf.copy() 
found = []
for i in datafr.index:
    if pd.isnull(datafr.loc[i, 'ID_ech']):
        found.append(i)
        datafr.loc[i, 'ID_ech'] = datafr.loc[i, 'ID']
if found: print(f"{len(found)} Nan found in 'ID_ech' and fixed")

74 Nan found in 'ID_ech' and fixed


In [147]:
for i in datafr.index:
    if re.search('FP 49 PROF', datafr.loc[i, 'ID_ech']):
        datafr.loc[i, 'ID_ech'] = 'FP49 PROF'
    elif re.search('FP 49 SUP', datafr.loc[i, 'ID_ech']):
        datafr.loc[i, 'ID_ech'] = 'FP49 SUP'

In [148]:
datafr = gen_id_from_ech(datafr, id_ech_col='ID_ech', suffixes=sufx, prefixes=prefx, capture_regex=id_reg)#, verbose=True)
mdf = datafr.copy()

In [149]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID','ID_ech','Date_mes'], dist_max=1)

In [150]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 1794


#### $\color{green}{\textbf{Lecture et fusion}}$

In [151]:
file1= csv_data_dir + 'profils_sols_donnees_forages/source_merge/source_Measures.csv' # 10
file2= csv_data_dir + 'database_Memoris3/source_merge/source_Measures.csv' # 12

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 42, columns : 12, Unique values on cols: {'ID': 30}
Rows : 1734, columns : 33, Unique values on cols: {'ID': 470}


(None, None)

In [152]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

0 Date_mes
0 ID
1 Date_mes
1 Date_for
1 ID
1 Residu_perte_feu
1 ID_ech


In [153]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID','Date_mes'], dist_max=1)

In [154]:
datafr = mdf.copy() 
found = []
for i in datafr.index:
    if pd.isnull(datafr.loc[i, 'ID_ech']):
        found.append(i)
        datafr.loc[i, 'ID_ech'] = datafr.loc[i, 'ID']
if found: print(f"{len(found)} Nan found in 'ID_ech' and fixed")

116 Nan found in 'ID_ech' and fixed


In [155]:
datafr = gen_id_from_ech(datafr, id_ech_col='ID_ech', suffixes=sufx, prefixes=prefx, capture_regex=id_reg)
mdf = datafr.copy()

In [156]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID','ID_ech','Date_mes'], dist_max=1)

Conflict values present. Please resolve this manually !


In [157]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'MS_x':list(conflict_df.index), 'pH_CaCl2_x':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [158]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 1978


#### $\color{green}{\textbf{Lecture et fusion}}$

In [159]:
file1= csv_data_dir + 'Container_phyto/source_merge/source_Measures.csv' # 0
file2= csv_data_dir + 'Siterem_Ext_Pilote/source_merge/source_Measures.csv' # 4

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> No coordinates
1 NaN lines dropped
df2 :  --> No coordinates
Rows : 5, columns : 14, Unique values on cols: {'ID': 'NA'}
Rows : 43, columns : 12, Unique values on cols: {'ID': 6}


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


(None, None)

In [160]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

0 Date_mes
0 ID_ech
0 Residu_perte_feu
1 Date_mes
1 ID_ech
1 ID


In [161]:
df2['ID_ech'] = df2['ID_ech'].astype('object')

In [162]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID_ech','Date_mes'], dist_max=1)

In [163]:
datafr = mdf.copy() 
found = []
for i in datafr.index:
    if pd.isnull(datafr.loc[i, 'ID_ech']):
        found.append(i)
        datafr.loc[i, 'ID_ech'] = datafr.loc[i, 'ID']
if found: print(f"{len(found)} Nan found in 'ID_ech' and fixed")

In [164]:
#datafr = gen_id_from_ech(datafr, id_ech_col='ID_ech', suffixes=sufx, prefixes=prefx, capture_regex=id_reg)
mdf = datafr.copy()

In [165]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID_ech','Date_mes'], dist_max=1)

In [166]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 2026


#### $\color{green}{\textbf{Lecture et fusion}}$

In [167]:
file1= csv_data_dir + 'Siterem_Pilote/source_merge/source_Measures.csv' #5
file2= csv_data_dir + 'Siterem_Result_Sol/source_merge/source_Measures.csv' #6

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> No coordinates
1 NaN lines dropped
df2 :  --> No coordinates
Rows : 135, columns : 11, Unique values on cols: {'ID': 18}
Rows : 58, columns : 13, Unique values on cols: {'ID': 22}


(None, None)

In [168]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

0 Date_mes
0 ID_ech
0 ID
1 Date_mes
1 ID_ech
1 ID


In [169]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID','ID_ech','Date_mes'], dist_max=1)

In [170]:
datafr = mdf.copy() 
found = []
for i in datafr.index:
    if pd.isnull(datafr.loc[i, 'ID_ech']):
        found.append(i)
        datafr.loc[i, 'ID_ech'] = datafr.loc[i, 'ID']
if found: print(f"{len(found)} Nan found in 'ID_ech' and fixed")

40 Nan found in 'ID_ech' and fixed


In [171]:
datafr = gen_id_from_ech(datafr, id_ech_col='ID_ech', suffixes=sufx, prefixes=prefx, capture_regex=id_reg)
mdf = datafr.copy()

In [172]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID','ID_ech','Date_mes'], dist_max=1)

In [173]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 2219


#### $\color{green}{\textbf{Lecture et fusion}}$

In [174]:
file1= csv_data_dir + 'donnees_terrain_2019/source_merge/source_Measures.csv' # 8
file2= csv_data_dir + 'observ_terrain/source_merge/source_Measures.csv' # 9

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

3 NaN lines dropped
df1 :  --> No coordinates
df2 :  --> No coordinates
Rows : 0, columns : 4, Unique values on cols: {'ID': 0}
Rows : 27, columns : 10, Unique values on cols: {'ID': 27}


(None, None)

In [175]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

0 Date_mes
0 ID
1 Date_mes
1 ID


In [176]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID','Date_mes'], dist_max=1)

In [177]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID','Date_mes'], dist_max=1)

In [178]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 2246


#### $\color{green}{\textbf{Lecture et fusion}}$

In [179]:
file1= csv_data_dir + 'result_sol_ext_pilote/source_merge/source_Measures.csv' # 11

df1 = create_df([file1])[0]
dataframe_viewer(df1, rows=3, un_val='ID', view=view)

df1 :  --> No coordinates
Rows : 53, columns : 13, Unique values on cols: {'ID': 19}


In [180]:
df_list = [df1]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

0 Date_mes
0 ID
0 ID_ech


In [181]:
data = df1
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID','ID_ech','Date_mes'], dist_max=1)

In [182]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 2260


####  $\color{red}{\textbf{Sauvegarde du jeu de données}}$

In [183]:
data_dict['Measure'] = dataset.copy()
dataframe_viewer(dataset, rows=3, un_val=['ID','ID_ech'])

Rows : 2260, columns : 44, Unique values on cols: {'ID': 556, 'ID_ech': 1730}


interactive(children=(IntSlider(value=3, description='rows', max=2260, min=3, readout=False), IntSlider(value=…

In [184]:
dataset.to_csv(save_dir + 'Measures.csv', index=False)

###  ====================================================

# Croisement et récupération de coordonnées

In [185]:
bh_cols = ['ID','Zone','Sous_zone','Emplacement','X','Y','Z','Zsol','Date_for','Type','Long_for','Long_pz',
           'Long_pz_sol','Diam_for','Diam_ext_pz','Diam_int_pz','Sect_crep','Ht_chbre','Ht_pz_sol','Refus',
           'Type_refus','Method','Societe','Etude', 'Resp_chantier','Rmq']

In [186]:
data_dict['Borehole'] = data_dict['Borehole'][bh_cols]

In [187]:
coi = ['ID','X','Y','Z','Date_for','Long_for','Diam_for']
bh_coords = data_dict['Borehole'][coi]

### Lithologies

In [188]:
data = data_dict['Litho'].copy()
dataset, conflict_df=data_merger(data, bh_coords, how='left', on=['ID'], dist_max=1)

Conflict values present. Please resolve this manually !


In [189]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'Long_for_x':list(conflict_df.index), 'Date_for_x':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [190]:
dataset.to_csv(save_dir + 'Cross_Litho-boreholes.csv', index=False)

### Samples

In [191]:
data = data_dict['Sample'].copy()
dataset, conflict_df=data_merger(data, bh_coords, how='left', on=['ID'], dist_max=1)

Conflict values present. Please resolve this manually !


In [192]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'Long_for_y':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [193]:
dataset.to_csv(save_dir + 'Cross_Samples-boreholes.csv', index=False)

### measures

In [194]:
data = data_dict['Measure'].copy()
dataset, conflict_df=data_merger(data, bh_coords, how='left', on=['ID', 'X'], dist_max=1)

In [196]:
dataset.to_csv(save_dir + 'Cross_Measures-boreholes.csv', index=False)